In [2]:
# Dependencies
from numpy import genfromtxt
from time import time
from datetime import datetime
from sqlalchemy import Column, Integer, Float, Date, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import Session
from sqlalchemy import Index
from sqlalchemy import MetaData
from sqlalchemy import Table
import csv
import pandas as pd
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine, inspect
from sqlalchemy import func

In [3]:
clean_measurement = "resources/withoutid_clean_measurement_df.csv"
clean_measurement_df = pd.read_csv(clean_measurement )
clean_measurement_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65.0
1,USC00519397,2010-01-02,0.00,63.0
2,USC00519397,2010-01-03,0.00,74.0
3,USC00519397,2010-01-04,0.00,76.0
4,USC00519397,2010-01-07,0.06,70.0


In [4]:
clean_station = "resources/withoutid_clean_station_df.csv"
clean_station_df = pd.read_csv(clean_station)

clean_station_df.head()

,station,station_name,latitude,longitude,elevation,location
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0,POINT(21.2716 -157.8168)
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6,POINT(21.4234 -157.8015)
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0,POINT(21.5213 -157.8374)
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9,POINT(21.3934 -157.9751)
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6,POINT(21.4992 -158.0111)


In [5]:
clean_station_df.dtypes

station          object
station_name     object
latitude        float64
longitude       float64
elevation       float64
location         object
dtype: object

In [6]:
Base = declarative_base()
class Measurement(Base):
    __tablename__ = 'measurement_table'
    __table_args__ = {'sqlite_autoincrement': True}
    __table_args__ = {'extend_existing': True}
    id = Column(Integer, primary_key=True, nullable=False)
    station = Column(String)
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)


engine = create_engine("sqlite:///hawaii_3.db")
Base.metadata.create_all(engine)

#Create the session
session = Session(bind=engine)

In [7]:
for i, row in clean_measurement_df.iterrows():
    
    record = Measurement(**{
        'station' : row.station,
        'date' : datetime.strptime(row.date, '%Y-%m-%d').date(),
        'prcp' : row.prcp,
        'tobs' : row.tobs
})

    session.add(record)
session.commit()

In [8]:
row

station    USC00516128
date        2017-08-23
prcp              0.45
tobs                76
Name: 18102, dtype: object

In [9]:

Base = declarative_base()
class Station(Base):
    __tablename__ = 'station_table'
    __table_args__ = {'sqlite_autoincrement': True}

    id = Column(Integer, primary_key=True, nullable=False)
    station = Column(String(225))
    station_name= Column(String(225)) 
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    location = Column(String(225))


engine = create_engine("sqlite:///hawaii_3.db")
Base.metadata.create_all(engine)

#Create the session
session = Session(bind=engine)

In [10]:
for i, row in clean_station_df.iterrows():
    record = Station(**{
         'station' : row.station,
         'station_name': row.station_name,
         'latitude': row.latitude,
         'longitude' : row.longitude,
         'elevation': row.elevation,
         'location': row.location
})
    print (row.station_name)
    session.add(record)
session.commit()

WAIKIKI 717.2, HI US
KANEOHE 838.1, HI US
KUALOA RANCH HEADQUARTERS 886.9, HI US
PEARL CITY, HI US
UPPER WAHIAWA 874.3, HI US
WAIMANALO EXPERIMENTAL FARM, HI US
WAIHEE 837.5, HI US
HONOLULU OBSERVATORY 702.2, HI US
MANOA LYON ARBO 785.2, HI US


In [11]:
row

station                          USC00516128
station_name    MANOA LYON ARBO 785.2, HI US
latitude                             21.3331
longitude                           -157.803
elevation                              152.4
location            POINT(21.3331 -157.8025)
Name: 8, dtype: object

In [12]:
Base = declarative_base()

In [13]:
engine = create_engine("sqlite:///hawaii_3.db")
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['measurement_table', 'station_table']

In [14]:
Measurement=Base.classes.measurement_table
Measurement

sqlalchemy.ext.automap.measurement_table

In [15]:
Station=Base.classes.station_table
Station

sqlalchemy.ext.automap.station_table

In [16]:
inspector = inspect(engine)
inspector.get_table_names()

['measurement_table', 'sqlite_sequence', 'station_table']

In [17]:
columns = inspector.get_columns('station_table')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station VARCHAR(225)
station_name VARCHAR(225)
latitude FLOAT
longitude FLOAT
elevation FLOAT
location VARCHAR(225)


In [18]:
from sqlalchemy import inspect

mapper = inspect(Measurement)
print(mapper)

mapped class measurement_table->measurement_table


In [19]:
from sqlalchemy import inspect

mapper = inspect(Station)
print(mapper)

mapped class station_table->station_table


In [20]:
Session = sessionmaker(bind=engine)
# Session is a class
session = Session()
# now session is a instance of the class Session

In [21]:
engine.execute('SELECT * FROM station_table LIMIT 5').fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0, 'POINT(21.2716 -157.8168)'),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6, 'POINT(21.4234 -157.8015)'),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0, 'POINT(21.5213 -157.8374)'),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9, 'POINT(21.3934 -157.9751)'),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6, 'POINT(21.4992 -158.0111)')]

In [22]:
engine.execute('SELECT * FROM measurement_table LIMIT 5').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0)]